In [5]:
# Packages for dataframe and stuff
import numpy as np
import pandas as pd

# For random forest stuff
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

#For visulizations
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [14]:
#Read in our trimmed CSV from the data cleanup step
data=pd.read_csv("../04-data_analysis/trimmed.csv")
#Remove rep column for now, probably should summarise (average) repeated measures later
data=data.drop('rep', axis=1)

In [41]:
#'Sample' column is the independent variable (samples) while the other columns (data) are dependent variables
y=np.array(data['sample'])
X=np.array(data.drop('sample', axis=1))

#Split data in to train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
#Build RF model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

ValueError: could not convert string to float: 'smoke'

In [18]:
accuracy=accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5833333333333334


In [25]:
#Low accuracy probably due to the fact that it's trying to predict individual sample groups intead of "smoke" vs "not-smoked"
#Need to make new column in CSV for smoke status and test on that instead
#print(data)
for index, row in data.iterrows():
    sample_value = str(row['sample'])  # Ensure string conversion
    if sample_value[0] == 's':
       data.loc[index, 'treatment'] = 'smoke'  # Assign to a specific row
    else:
       data.loc[index, 'treatment'] = 'control'

In [27]:
#print(data)

In [47]:
#Try again
y=np.array(data['treatment'])
X=np.array(data.drop(['sample','treatment'], axis=1))

#Split data in to train and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
#Build RF model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [49]:
y_pred=rf.predict(X_test)

In [50]:
accuracy=accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9166666666666666


In [51]:
#MUUUUCH better
#Now need boruta testing to weed out features
from boruta import BorutaPy

In [54]:
#Have to change definitions because BorutaPy uses an outdated version of numpy
np.int = np.int32
np.float = np.float64
np.bool=np.bool_

#Define feature selection method
selector=BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [55]:
#Find relevant features
selector.fit(X,y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	55450
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	731
Rejected: 	54719
Iteration: 	9 / 100
Confirmed: 	99
Tentative: 	632
Rejected: 	54719
Iteration: 	10 / 100
Confirmed: 	99
Tentative: 	632
Rejected: 	54719
Iteration: 	11 / 100
Confirmed: 	99
Tentative: 	632
Rejected: 	54719
Iteration: 	12 / 100
Confirmed: 	110
Tentative: 	458
Rejected: 	54882
Iteration: 	13 / 100
Confirmed: 	110
Tentative: 	458
Rejected: 	54882
Iteration: 	14 / 100
Confirmed: 	110
Tentative: 	458
Rejected: 	54882
Iteration: 	15 / 100
Confirmed: 	110
Tentative: 	458
Rejected:

BorutaPy(estimator=RandomForestClassifier(n_estimators=181,
                                          random_state=RandomState(MT19937) at 0x7F10A62A2740),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7F10A62A2740, verbose=2)

In [57]:
selector.ranking_

array([34554, 34554,   581, ..., 34554,  8506, 34554], dtype=int32)

In [58]:
X_filtered = selector.transform(X)

In [59]:
#Split data in to train and testing sets using the new feature list from boruta testing
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y, test_size=0.2)

In [60]:
#Build RF model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [61]:
y_pred=rf.predict(X_test)

In [62]:
accuracy=accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [63]:
print(X_filtered)

[[ 44585.4    118502.4    611233.6    ...  13335.94    10757.63
       0.    ]
 [ 21627.87    82145.65   461915.5    ...      0.          0.
       0.    ]
 [ 30635.64   101212.5    580755.     ...      0.          0.
       0.    ]
 ...
 [ 27656.08    81016.56   491885.8    ...   6553.5996      0.
       0.    ]
 [ 49249.81   121152.     649991.     ...  16893.96    12618.99
    8465.199 ]
 [ 27833.5     83168.84   499433.1    ...      0.          0.
       0.    ]]
